# Phase 2A: Interactive Visualizations for Multi-Asset OHLCV Data

Interactive visualizations for all assets exported from Phase 1B across multiple asset types:
- **Traditional Commodities** (Gold, Silver, Oil, Natural Gas): DeFi perpetuals vs TradFi futures
- **Traditional Equities** (AAPL, GOOGL, MSFT, NVDA, TSLA, etc.): DeFi perpetuals vs TradFi stocks
- **Crypto Coins** (BTC, ETH, SOL, LINK, ADA): DEX perpetuals vs CEX spot markets

**Terminology:**
- For traditional assets: **DeFi** (decentralized perpetuals) vs **TradFi** (traditional markets)
- For crypto assets: **DEX** (decentralized perpetuals) vs **CEX** (centralized exchange spot)

**Visualizations (Organized by Asset Type):**
For each asset type, the notebook presents:
1. **Overview Dashboard** - Summary statistics including volume metrics, price correlation, and data overlap
2. **Volume Comparison** - Notional volume charts showing DeFi/DEX vs TradFi/CEX trading activity

---

## Implementation Notes

This notebook uses utility functions from `utils.visualization` for clean, maintainable code:
- `load_all_assets()` - Loads all asset data from Phase 1B output
- `create_overview_dashboard()` - Generates summary statistics tables with customizable date filtering
- `plot_assets_by_type()` - Creates volume comparison plots for all assets of a given type

**Key Features:**
- Parameterized functions automatically adapt labels based on asset type (DeFi/TradFi vs DEX/CEX)
- Date filtering support (e.g., crypto assets filtered to July 2024 onwards)
- Dual y-axis volume plots for better visualization across different volume scales
- Consistent analysis across all asset types with minimal code repetition

In [ ]:
import os

from utils import (
    load_all_assets,
    create_overview_dashboard,
    plot_assets_by_type,
)

PHASE_1B_DIR = os.path.join("output", "Phase 1B")

## Load All Assets from Phase 1B

In [ ]:
# Load all assets from Phase 1B output directory
assets, assets_by_type = load_all_assets(PHASE_1B_DIR)
print(f"\nTotal assets loaded: {len(assets)}")

# Visualizations by Asset Type

For each asset type, we'll create:
1. **Overview Dashboard** - Summary statistics for all assets in the type
2. **Volume Comparison** - Notional volume charts for each asset

In [4]:
# Get sorted asset types for reference
sorted_asset_types = sorted(assets_by_type.keys())
print("Available asset types:", sorted_asset_types)

Available asset types: ['Crypto Coin', 'Traditional Commodity', 'Traditional Equity']


---
# Crypto Coin Assets
## DEX Perpetuals vs CEX Spot Markets
---

In [ ]:
# Create overview dashboard for Crypto Coins (filtered to July 2024 onwards)
asset_type = "Crypto Coin"
asset_names = assets_by_type[asset_type]

overview_df = create_overview_dashboard(
    assets, 
    asset_type, 
    asset_names, 
    date_filter="2024-07-01"
)
display(overview_df)

In [ ]:
# Plot volume comparisons for all Crypto Coin assets (from July 2024)
plot_assets_by_type(
    assets,
    asset_type="Crypto Coin",
    asset_names=assets_by_type["Crypto Coin"],
    date_filter="2024-07-01",
    use_dual_axis=True
)

---
# Traditional Commodity Assets
## DeFi Perpetuals vs TradFi Futures
---

In [ ]:
# Create overview dashboard for Traditional Commodities
asset_type = "Traditional Commodity"
asset_names = assets_by_type[asset_type]

overview_df = create_overview_dashboard(assets, asset_type, asset_names)
display(overview_df)

In [ ]:
# Plot volume comparisons for all Traditional Commodity assets
plot_assets_by_type(
    assets,
    asset_type="Traditional Commodity",
    asset_names=assets_by_type["Traditional Commodity"],
    use_dual_axis=True
)

---
# Traditional Equity Assets
## DeFi Perpetuals vs TradFi Stocks
---

In [ ]:
# Create overview dashboard for Traditional Equities
asset_type = "Traditional Equity"
asset_names = assets_by_type[asset_type]

overview_df = create_overview_dashboard(assets, asset_type, asset_names)
display(overview_df)

In [ ]:
# Plot volume comparisons for all Traditional Equity assets
plot_assets_by_type(
    assets,
    asset_type="Traditional Equity",
    asset_names=assets_by_type["Traditional Equity"],
    use_dual_axis=True
)